In [1]:
import vaex as vx
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import scipy as sc
from scipy.interpolate import PchipInterpolator as minterp
import scipy.stats as stats
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm import tqdm

import os
import sys
import re
from numba import jit
from collections import Counter
from collections import namedtuple
from collections import defaultdict

from sklearn.covariance import OAS
from sklearn.covariance import GraphicalLassoCV
from sklearn.covariance import LedoitWolf
from sklearn.covariance import EmpiricalCovariance

from _helpers import _helpers as hlp

# ERV differentiation

Two main research questions:
1. Are ERV's differentially expressed in SCC versus AC?
2. Are ERV's involved in immune surveillance in lung cancer

We have to 
* identify ERV's in Adeno and SCC
* identify granzy mand perforin expression per subtype
* identify IFN gene expression per subtype
* silencing of ERV by methylation in normal tissue



In [2]:
max_nans_perc = 0.1

In [3]:
os.chdir('/media/koekiemonster/DATA-FAST/genetic_expression/hackathon_2/Lung/')

In [4]:
dd = {}
dd['RNAex'] = vx.open('Lung_GeneExpression.hdf5')
dd['mutation'] = vx.open('Lung_Mutation.hdf5')
dd['methylation'] = vx.open('Lung_Methylation.hdf5')
dd['methylation_meta'] = vx.open('HumanMethylation450_meta.hdf5')
dd['CNV'] = vx.open('Lung_CNV.hdf5')

meta = pd.read_csv('Lung_Phenotype_Metadata.txt', sep='\t')
meta = meta.loc[~pd.isna(meta.Diagnosis)]
meta = meta[~meta.SampleID.duplicated(keep='last')]
meta.set_index('SampleID', inplace=True)

In [5]:
meta.columns

Index(['PatientID', 'Diagnosis', 'Sample Type', 'Gender', 'Ethnicity',
       'Age At Diagnosis (Days)', 'Age At Diagnosis (Years)', 'Vital Status ',
       'Overall Survival Status', 'Time To Overall Survival (Days)',
       'Time To Overall Survival From Treatment (Days)', 'Relapse Status',
       'Time To Relapse From Diagnosis (Days)', 'Event Status',
       'Time To Event From Diagnosis (Days)', 'Karnofsky Score',
       'Smoking Status', 'Pack Years', 'Histological Type', 'Morphology',
       'Site Of Resection', 'Tumor Location', 'Anatomic Location In Lung',
       'Tumor Stage', 'T-stage', 'N-stage', 'M-stage', 'R-stage',
       'Cancer Status After Resection', 'Drug Therapy Type', 'Drug Name',
       'Time To Drug Therapy (Days)', 'Radiation Therapy',
       'Time To Radiation Therapy (Days)', 'Reponse To Therapy',
       'Time To Therapy (Days)', 'Therapy Ongoing', 'New Tumor Event',
       'New Tumor Event Type', 'Time To New Tumor Event (Days)'],
      dtype='object')

In [6]:
erv_genes = [_gene for _gene in dd['RNAex'].Gene.unique() if 'ERV' in _gene]
erv_genes += [_gene for _gene in dd['methylation'].Gene.unique() if 'ERV' in _gene]
erv_genes = list(set(erv_genes))

In [7]:
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5942792/
neutro_gene_list = ['DEFA4', 'DEFA3', 'MMP8', 'CEACAM6', 'CEACAM8', 'LTF', 'MPO', 'ARG1', 'MSHA3']
neutro_genes = [_gene for _gene in dd['RNAex'].Gene.unique() if _gene in neutro_gene_list]
neutro_genes += [_gene for _gene in dd['methylation'].Gene.unique() if _gene in neutro_gene_list]
neutro_genes = list(set(neutro_genes))
# https://www.jimmunol.org/content/175/12/7837.long, http://www.informatics.jax.org/go/term/GO:0033077
t_cell_gene_list = ['CXCR5', 'CXCR3', 'CCR3', 'CCR5', 'CRTH2', 'CCR6', 'CCR7', 'CD3D', 'CD3E', 'CD3G', 'ATG5']
tcell_genes = [_gene for _gene in dd['RNAex'].Gene.unique() if _gene in t_cell_gene_list]
tcell_genes += [_gene for _gene in dd['methylation'].Gene.unique() if _gene in t_cell_gene_list]
tcell_genes = list(set(tcell_genes))
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3219766/, https://www.sciencedirect.com/science/article/pii/S1521690X17300301?via%3Dihub
thyroid_gene_list = ['PDE8B', 'DIO1', 'DIO2', 'CAPZB', 'TSHR', 'THRA', 'THRB', 'MCT8', 'MCT10', 'NTCP', 
                    'SLC10A1', 'SLC10A2', 'SLC16A10', 'LAT1', 'LAT2', 'OATP1A2', 'OATP1B1', 'OATP1B3', 'OATP1C1']
thyroid_genes = [_gene for _gene in dd['RNAex'].Gene.unique() if _gene in thyroid_gene_list]
thyroid_genes += [_gene for _gene in dd['methylation'].Gene.unique() if _gene in thyroid_gene_list]
thyroid_genes = list(set(tcell_genes))
# https://www.sciencedaily.com/releases/2016/08/160822125438.htm, 
thymus_gene_list = ['FOXN1']

In [8]:
# external
os.chdir('/media/koekiemonster/home/bramiozo/DEV/GIT/RexR/_hackathon2018/_meta_data/gene_types/')
for idx, _file in enumerate(os.listdir(".")):
    if _file.split(".")[1] in ['txt']: 
        _genes = pd.read_csv(_file, sep="\t", index_col=False)
        _genes['gene_class'] = _file.split(".")[0]
        try:
            ext_genes = pd.concat([ext_genes, _genes[['Symbol', 'gene_class']]]) if idx>0 else _genes[['Symbol', 'gene_class']]
        except Exception as e:
            print('Exception:',e, '\n', 'For file:', _file)

In [9]:
for idx, _class in enumerate(ext_genes.gene_class.unique()):
    tmp = ext_genes.loc[ext_genes.gene_class==_class].drop_duplicates()
    cleaned_genes = pd.concat([cleaned_genes, tmp]) if idx>0 else tmp
cleaned_genes.reset_index(drop=True, inplace=True)
ext_genes = cleaned_genes

In [10]:
thyroid = pd.DataFrame(data=[(_gen, 'thyroid') for _gen in thyroid_genes], columns=['Symbol', 'gene_class'])
tcell = pd.DataFrame(data=[(_gen, 'tcell') for _gen in tcell_genes], columns=['Symbol', 'gene_class'])
thymus = pd.DataFrame(data=[(_gen, 'thymus') for _gen in thymus_gene_list], columns=['Symbol', 'gene_class'])
neutro = pd.DataFrame(data=[(_gen, 'neutro') for _gen in neutro_genes], columns=['Symbol', 'gene_class'])
erv = pd.DataFrame(data=[(_gen, 'erv') for _gen in erv_genes], columns=['Symbol', 'gene_class'])
ext_genes = pd.concat([ext_genes, thyroid, tcell, erv, thymus, neutro])
ext_genes['Symbol'] = ext_genes.Symbol.str.upper()
ext_genes = ext_genes.groupby('Symbol')['gene_class'].apply(lambda x: "|".join(x)).reset_index()

In [11]:
gene_list = ext_genes.Symbol.unique().tolist()

In [23]:
#ext_genes.to_csv("selected_genes.csv", sep="\t", index=False)

In [ ]:
df_erv = {}
for _key in dd:  
    d = dd[_key]
    print("Processing:", _key)
    if 'Gene' in d.columns:
        temps = []
        for _gene in tqdm(gene_list):
            temps.append(d[d.Gene.str.upper() == _gene].to_pandas_df())            
        df_erv[_key] = pd.concat(temps, axis=0)
var_cols = {}

  0%|          | 0/995 [00:00<?, ?it/s]

In [ ]:
probe_map = df_erv['methylation'][['probeID', 'Chr', 'Start', 'Stop', 'Strand', 'Gene', 'Relation_CpG_Island']].reset_index().drop('index', axis=1)
df_erv['methylation'] = df_erv['methylation'].drop(['Chr', 'Start', 'Stop', 'Strand', 'Gene', 'Relation_CpG_Island'], axis=1)
df_erv['methylation'].set_index('probeID', inplace=True)
df_erv['methylation'] = df_erv['methylation'].transpose()

max_nans = int(max_nans_perc*df_erv['methylation'].shape[0])
df_erv['methylation'].index = [re.sub(r'_', '-', _old_key) for _old_key in df_erv['methylation'].index.tolist()]
df_erv['methylation'] = df_erv['methylation'].loc[:, df_erv['methylation'].columns[df_erv['methylation'].isna().sum()\
                                                                                   <max_nans].tolist()]
var_cols['methylation'] = df_erv['methylation'].columns.tolist()

In [ ]:
exp_map = df_erv['RNAex'][['Gene', 'Chr', 'Start', 'Stop', 'Strand']].reset_index().drop('index', axis=1)
df_erv['RNAex'] = df_erv['RNAex'].drop(['Chr', 'Start', 'Stop', 'Strand'], axis=1)
df_erv['RNAex'].set_index('Gene', inplace=True)
df_erv['RNAex'] = df_erv['RNAex'].transpose()

max_nans = int(max_nans_perc*df_erv['RNAex'].shape[0])
df_erv['RNAex'].index = [re.sub(r'_', '-', _old_key) for _old_key in df_erv['RNAex'].index.tolist()]
df_erv['RNAex'] = df_erv['RNAex'].loc[:, df_erv['RNAex'].columns[df_erv['RNAex'].isna().sum()\
                                                                                   <max_nans].tolist()]
var_cols['RNAex'] = df_erv['RNAex'].columns.tolist()

In [ ]:
df_erv['mutation'] = df_erv['mutation'][df_erv['mutation'].Effect!='upstream_gene_variant']
mut_map = df_erv['mutation'][['Amino_Acid_Change', 'Gene', 'Chr', 'Start', 'Stop', 'Ref', 'Alt']].reset_index().drop('index', axis=1)
df_erv['mutation'] = df_erv['mutation'].drop(['Gene', 'Chr', 'Start', 'Stop', 'Ref', 'Alt'], axis=1)
df_erv['mutation'] = df_erv['mutation'].reset_index().drop('index', axis=1)
df_erv['mutation'] = df_erv['mutation'].rename(index=str, columns={'Sample_ID': 'SampleID'})
df_erv['mutation'].set_index('SampleID', inplace=True)
var_cols['mutation'] = df_erv['mutation'].columns.tolist()

In [ ]:
cnv_map = df_erv['CNV'][['Gene', 'Chr', 'Start', 'Stop', 'Strand']]
df_erv['CNV'] = df_erv['CNV'].drop(['Chr', 'Start', 'Stop', 'Strand'], axis=1)
df_erv['CNV'] = df_erv['CNV'].set_index('Gene')
df_erv['CNV'] = df_erv['CNV'].transpose()
df_erv['CNV'].index = [re.sub(r'_', '-', _old_key) for _old_key in df_erv['CNV'].index.tolist()]
var_cols['CNV'] = df_erv['CNV'].columns.tolist()

In [ ]:
meta_cols = ['Gender', 'Diagnosis', 'Age At Diagnosis (Years)', 'Overall Survival Status', 'Pack Years', 'Smoking Status']
meta_cols = meta_cols + ['New Tumor Event', 'Radiation Therapy', 'Reponse To Therapy', 'Drug Therapy Type']

stage_map = {'stage i': 'stage 1', 'stage ia': 'stage 1', 'stage ib': 'stage 1',
             'stage ii': 'stage 2', 'stage iia': 'stage 2', 'stage iib': 'stage 2',
             'stage iii': 'stage 3','stage iiia': 'stage 3', 'stage iiib': 'stage 3',
             'stage iv' : 'stage 4'}
meta['Stage'] = meta['Tumor Stage'].map(stage_map)
########
smoke_map = {'Current Reformed Smoker < or = 15 yrs': 'reformed', 
             'Current Reformed Smoker for > 15 yrs': 'reformed',
             'Current Reformed Smoker, Duration Not Specified': 'reformed',
             'Current Smoker': 'current',
             'Lifelong Non-Smoker': 'non-smoker'}
meta['Smoking'] = meta['Smoking Status'].map(smoke_map)
########
response_map = {'Progressive Disease': 'no',
                'Complete Remission/Response': 'yes',
                'Stable Disease': 'no',
                'Partial Remission/Response': 'yes'}
meta['Response'] = meta['Reponse To Therapy'].map(response_map)
meta_cols = list(set(meta_cols + ['Stage', 'Smoking', 'Response']))

for _key in df_erv:
    df_erv[_key] = df_erv[_key].join(meta[meta_cols])

In [ ]:
df_erv.keys()

# Check differentiation per feature

In [ ]:
#probe_map # probID -> Gene
#ext_genes # Symbol -> gene_class

## Methylation

In [ ]:
len(var_cols['methylation'])

In [ ]:
i=9999
meta_col = 'Diagnosis' # 
meta_vals = df_erv['methylation'][meta_col].unique().tolist()
val_series = []
probe = var_cols['methylation'][i]
gene = probe_map.loc[probe_map.probeID==probe]['Gene'].values[0]
gene_class = ext_genes.loc[ext_genes.Symbol==gene]['gene_class'].values[0]
for _meta_col_val in meta_vals:
    if pd.isna(_meta_col_val)==False:
        x = df_erv['methylation'].loc[df_erv['methylation'][meta_col]==_meta_col_val, probe]
        x.plot.hist(bins=20, alpha=0.5, density=True, label=_meta_col_val)
        x.plot.kde(color='black', label='')
        val_series.append(x)
plt.title(meta_col+' , '+var_cols['methylation'][i]+' ,  '+gene+' , '+gene_class)
plt.legend()
plt.xlim(0., 1.1)

In [ ]:
# Wasserstein distance per gene class
dist_list_diag = []
for _probe in tqdm(var_cols['methylation']):
    v1 = df_erv['methylation'].loc[df_erv['methylation']['Diagnosis']=='Lung Adenocarcinoma', _probe]
    v2 = df_erv['methylation'].loc[df_erv['methylation']['Diagnosis']=='Lung Squamous Cell Carcinoma', _probe]
    
    try:
        cd1 = _cdf_distanceB(v1, v2)
        cd2 = _cdf_distanceG(v1, v2)
    except ValueError as e:
        v1 = np.isfinite(v1)
        v2 = np.isfinite(v2)
        cd1 = _cdf_distanceB(v1, v2)
        cd2 = _cdf_distanceG(v1, v2)
    dist_list_diag.append({'probe': _probe, 
                      'Bdist1': cd1[0],
                      'Bdist2': cd1[1],
                      'Bdist3': cd1[2],
                      'Gdist1': cd2[0], 
                      'Gdist2': cd2[1],
                      'Gdist3': cd2[2],
                      'wass1' : stats.wasserstein_distance(v1,v2),
                      'wass2' : stats.energy_distance(v1,v2),
                      'statdistKS': _scaled_stat_distance(v1,v2)
                      })

In [ ]:
dist_list_response = []
for _probe in tqdm(var_cols['methylation']):
    v1 = df_erv['methylation'].loc[df_erv['methylation']['Response']=='yes', _probe]
    v2 = df_erv['methylation'].loc[df_erv['methylation']['Response']=='no', _probe]
    
    try:
        cd1 = _cdf_distanceB(v1, v2)
        cd2 = _cdf_distanceG(v1, v2)
    except ValueError as e:
        v1 = np.isfinite(v1)
        v2 = np.isfinite(v2)
        cd1 = _cdf_distanceB(v1, v2)
        cd2 = _cdf_distanceG(v1, v2)
    dist_list_response.append({'probe': _probe, 
                      'Bdist1': cd1[0],
                      'Bdist2': cd1[1],
                      'Bdist3': cd1[2],
                      'Gdist1': cd2[0], 
                      'Gdist2': cd2[1],
                      'Gdist3': cd2[2],
                      'wass1' : stats.wasserstein_distance(v1,v2),
                      'wass2' : stats.energy_distance(v1,v2),
                      'statdistKS': _scaled_stat_distance(v1,v2)
                      })

In [ ]:
dist_list_df = pd.DataFrame(dist_list_response)
dist_list_df = dist_list_df.merge(probe_map[['probeID', 'Gene']], how='inner', left_on='probe', right_on='probeID')
dist_list_df = dist_list_df.merge(ext_genes, how='inner', left_on='Gene', right_on='Symbol')
agg_by_gene = dist_list_df.groupby('Gene').mean()

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(24,18))
dist_list_df.sort_values(by='statdistKS', ascending=False)[:1000].groupby('gene_class').size().plot.bar(ax=ax[0,0])
dist_list_df.sort_values(by='statdistKS', ascending=True)[:1000].groupby('gene_class').size().plot.bar(ax=ax[0,1])
sns.scatterplot(data=dist_list_df, x='wass1', y='statdistKS', hue='gene_class', alpha=0.3, ax=ax[1,0])
sns.scatterplot(data=dist_list_df, x='wass1', y='wass2', hue='gene_class', alpha=0.3, ax=ax[1,1])
plt.suptitle("Lung cancer response methylation differentiation")

# Sample to sample similarity

## Cross correlation
* quantile transformation
* cross correlations -> distribution



## Graph analysis

### graph clustering

### graph embedding

# Statistical properties of target groups

In [ ]:
'''
Univariated intra-distribution characterisations:
    * [x] skewness
    * [x] kurtosis
    * normality test
        * [x] q-qnorm metrics
        * [x] Shapiro-Wilk
        * [x] Anderson-Darling
        * [x] Jarque-Bera
        * [x] Gel-Gastwirth test, see https://ideas.repec.org/a/eee/ecolet/v99y2008i1p30-32.html
        * [x] KS-goodness of fit
'''

'''
 function add_stat_agg(df, agg_val='Diagnosis')
'''

stat_cat = defaultdict(dict)
stat_cat['methylation']['skewness'] = df_erv['methylation']\
                            .loc[:, var_cols['methylation']+['Diagnosis']]\
                            .groupby('Diagnosis')\
                            .apply(lambda x: _stanmom(x, mom=3, logscale=True)).transpose()

stat_cat['methylation']['kurtosis'] = df_erv['methylation']\
                            .loc[:, var_cols['methylation']+['Diagnosis']]\
                            .groupby('Diagnosis')\
                            .apply(lambda x: _stanmom(x, mom=4, logscale=True)).transpose()

stat_cat['methylation']['qqnorm'] = df_erv['methylation'].loc[:, var_cols['methylation']+['Diagnosis']]\
                     .groupby('Diagnosis')\
                     .apply(lambda x: _qq(x, minkowski=2)).reset_index(level=1)
stat_cat['methylation']['qqnorm'] = stat_cat['methylation']['qqnorm'].drop('level_1', axis=1)
stat_cat['methylation']['qqnorm'] = stat_cat['methylation']['qqnorm'].transpose()

stat_cat['methylation']['mean'] = df_erv['methylation'].loc[:, var_cols['methylation']+['Diagnosis']]\
                     .groupby('Diagnosis')\
                     .mean().transpose()

stat_cat['methylation']['median'] = df_erv['methylation'].loc[:, var_cols['methylation']+['Diagnosis']]\
                     .groupby('Diagnosis')\
                     .median().transpose()

stat_cat['methylation']['q75'] = df_erv['methylation'].loc[:, var_cols['methylation']+['Diagnosis']]\
                     .groupby('Diagnosis')\
                     .quantile(0.75).transpose()

stat_cat['methylation']['q25'] = df_erv['methylation'].loc[:, var_cols['methylation']+['Diagnosis']]\
                     .groupby('Diagnosis')\
                     .quantile(0.25).transpose()

stat_cat['methylation']['AD'] = df_erv['methylation'].loc[:, var_cols['methylation']+['Diagnosis']]\
                     .groupby('Diagnosis')\
                     .apply(lambda x: stat_test_arr(x, test='AD')).reset_index(level=1)
stat_cat['methylation']['AD'] = stat_cat['methylation']['AD'].drop('level_1', axis=1)
stat_cat['methylation']['AD'] = stat_cat['methylation']['AD'].transpose()

stat_cat['methylation']['SW'] = df_erv['methylation'].loc[:, var_cols['methylation']+['Diagnosis']]\
                     .groupby('Diagnosis')\
                     .apply(lambda x: stat_test_arr(x, test='SW')).reset_index(level=1)
stat_cat['methylation']['SW'] = stat_cat['methylation']['SW'].drop('level_1', axis=1)
stat_cat['methylation']['SW'] = stat_cat['methylation']['SW'].transpose()


stat_cat['methylation']['JB'] = df_erv['methylation'].loc[:, var_cols['methylation']+['Diagnosis']]\
                     .groupby('Diagnosis')\
                     .apply(lambda x: stat_test_arr(x, test='JB')).reset_index(level=1)
stat_cat['methylation']['JB'] = stat_cat['methylation']['JB'].drop('level_1', axis=1)
stat_cat['methylation']['JB'] = stat_cat['methylation']['JB'].transpose()

stat_cat['methylation']['RJB'] = df_erv['methylation'].loc[:, var_cols['methylation']+['Diagnosis']]\
                     .groupby('Diagnosis')\
                     .apply(lambda x: _rjb(x, C1=6, C2=64)).transpose()

In [ ]:
'''
Univariate inter-distribution distance:    
        * _information_change: KL, Cross, Shannon
        * Kolmogorov-Smirnov (also normality test)
        * Pearson's chi-squared test (only if all vars are binned and categorised)
        * Mann-Whitney U
        * Brunner Munzel
        * Bonett-Seier test
        * Kruskall Wallis
        * FDR-BH-ANOVA
        * total variation distance
        * Hellinger distance
        * Bhattacharyya distance
        * KS goodness of fit
        * [x] q-q distances (CvM, nth-Wasserstein, etc.)
        * [x] CDF distance 
        
    
        function add_inter_dist(df, split_cols, split_val='Diagnosis')
'''



In [ ]:
'''
also see: https://towardsdatascience.com/select-features-for-omics-integration-511390b7e7fd
inter-class separation scores:
 - ElasticNet - binomial
 - LR
 - PLS-DA
 - LDA
 - bi-GAM
'''

In [ ]:
# we extract the weights as weights for the similarity check later

In [ ]:
'''
    similarity -> dissimilarity:
    * d = 1 - s
    * d = sqrt(s(x,x)+s(y,y)-2s(x,y))
    * d= -log(s(x,y))
'''

'''
    dissimilarity -> similarity:
    * s = exp(-d(x,y)/sigma)
    * s = 1/(1+d(x,y))
    * s = 1/2*(d^2(x,c) + d^2(y,c)-d^2(x,y))
    
'''

In [ ]:
v1 = np.array(df_erv['methylation'][var_cols['methylation']].iloc[41,:])
v2 = np.array(df_erv['methylation'][var_cols['methylation']].iloc[151,:])
indexsort = np.argsort(v1)

fig, ax = plt.subplots(ncols=2, figsize=(18,7))
ax[0].plot(v1[indexsort])
ax[0].plot(v2[indexsort])
ax[1].plot(v1[indexsort], v2[indexsort])

print(_corr(v1, v2, fun=sc.stats.spearmanr), _corr(v1, v2, fun=sc.stats.pearsonr), _corr(v1, v2, fun=sc.stats.f_oneway))
w = np.ones(shape=v1.shape)
print(_wcorr(v1, v2, w))
print(_continuous_sim(v1, v2, fun=sc.spatial.distance.cosine, centered=True, w=w))

In [ ]:
'''
Pairwise distance:
    * Manhattan distance (only if all vars are binned and categorised)
    * Euclidean distance
    * Pearson  distance To be consistent with data protection laws, we’re asking you to take a moment to review key points of our Privacy Policy, which covers all Google services and describes how we use data and what options you have. We'll need you to do this today.
NL

    * Kendall distance 
    * Spearman distance
    * 1-Cosine distance (pseudometric)
    * Mahalanobis distance
'''

In [ ]:
# get inter-dataset feature-feature similarity 

# get sample-sample similarity
# cluster samples 

# use NN to find similar patient clusters https://github.com/spotify/annoy

## make N s-s-s bins 


# R: mixOmics, https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1005752
# DIABLO: https://www.biorxiv.org/content/10.1101/067611v2
# Multiscale, multifactorial response network (MMRN)
# https://www.nature.com/articles/s41467-019-08794-x.pdf
# https://www.sciencedirect.com/science/article/pii/S2452310018301197#undfig1
# https://www.cell.com/cell-systems/fulltext/S2405-4712(17)30548-3
# https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-12-253

In [ ]:
# what combination of dimensions makes samples from different target groups most dissimilar?
# two target groups: AdenoCarcinoma and SquamousCellCarcinoma
# maximize wasserstein distance
# maximize information gain.

## Check class/survival differentiation per feature combination 


Another way to check differentiation between groups is to compare the survival curve changes per feature. 
We do this by splitting the groups on the feature medians.

## Check differentiation class/survival with/without ERV mutations

## Weighted, supervised UMAP for identifying clusters

## Hausdorff distance between sample groups
See the [wiki](https://en.wikipedia.org/wiki/Hausdorff_distance).

SciPy has a function readily available, see [directed Hausdorff](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.directed_hausdorff.html)


## Check activation of neutrofil/T-cell genes in relation to efficacy of immunotherapy

In [ ]:
result_map = {'Complete Remission/Response': 1, 
              'Stable Disease': 0, 
              'Progressive Disease': 0,
              'Partial Remission/Response': 1}

neutro_probes = probe_map.loc[probe_map.Gene.isin(neutro_gene_list), 'probeID'].unique().tolist()
tcell_probes = probe_map.loc[probe_map.Gene.isin(tcell_genes), 'probeID'].unique().tolist()
erv_probes = probe_map.loc[probe_map.Gene.isin(erv_genes), 'probeID'].unique().tolist()

neutro_probes  = set(neutro_probes) & set(df_erv['methylation'].columns.tolist())
tcell_probes = set(tcell_probes) & set(df_erv['methylation'].columns.tolist())
erv_probes = set(erv_probes) & set(df_erv['methylation'].columns.tolist())
df_erv['methylation']['RTT'] = df_erv['methylation']['Reponse To Therapy'].map(result_map)

neutro_rna =  set(neutro_gene_list) & set(df_erv['RNAex'].columns.tolist())
tcell_rna = set(tcell_genes) & set(df_erv['RNAex'].columns.tolist())
erv_rna = set(erv_genes) & set(df_erv['RNAex'].columns.tolist())

df_erv['RNAex']['RTT'] = df_erv['RNAex']['Reponse To Therapy'].map(result_map)

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(18,5))
df_erv['methylation'].loc[df_erv['methylation'].Diagnosis=='Lung Adenocarcinoma', erv_probes]\
                                                    .mean(axis=0).plot.hist(bins=20, histtype='step', ax=ax[0], label='Adeno')
df_erv['methylation'].loc[df_erv['methylation'].Diagnosis=='Lung Squamous Cell Carcinoma', erv_probes]\
                                                    .mean(axis=0).plot.hist(bins=20, histtype='step', ax=ax[0], label='SCC')

df_erv['methylation'].loc[df_erv['methylation'].Diagnosis=='Lung Adenocarcinoma', neutro_probes]\
                                                    .mean(axis=0).plot.hist(bins=20, histtype='step', ax=ax[1], label='Adeno')
df_erv['methylation'].loc[df_erv['methylation'].Diagnosis=='Lung Squamous Cell Carcinoma', neutro_probes]\
                                                    .mean(axis=0).plot.hist(bins=20, histtype='step', ax=ax[1], label='SCC')

df_erv['methylation'].loc[df_erv['methylation'].Diagnosis=='Lung Adenocarcinoma', tcell_probes]\
                                                    .mean(axis=0).plot.hist(bins=20, histtype='step', ax=ax[2], label='Adeno')
df_erv['methylation'].loc[df_erv['methylation'].Diagnosis=='Lung Squamous Cell Carcinoma', tcell_probes]\
                                                    .mean(axis=0).plot.hist(bins=20, histtype='step', ax=ax[2], label='SCC')

ax[0].set_title('Mean probe values for ERV')
ax[1].set_title('Mean probe values for neutrophils')
ax[2].set_title('Mean probe values for t-cells')
ax[0].legend(loc=2)
ax[1].legend(loc=2)
ax[2].legend(loc=2)

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(18,5))
df_erv['RNAex'].loc[df_erv['RNAex'].Diagnosis=='Lung Adenocarcinoma', erv_rna]\
                                                    .mean(axis=0).plot.hist(bins=5, histtype='step', ax=ax[0], label='Adeno')
df_erv['RNAex'].loc[df_erv['RNAex'].Diagnosis=='Lung Squamous Cell Carcinoma', erv_rna]\
                                                    .mean(axis=0).plot.hist(bins=5, histtype='step', ax=ax[0], label='SCC')

df_erv['RNAex'].loc[df_erv['RNAex'].Diagnosis=='Lung Adenocarcinoma', neutro_rna]\
                                                    .mean(axis=0).plot.hist(bins=5, histtype='step', ax=ax[1], label='Adeno')
df_erv['RNAex'].loc[df_erv['RNAex'].Diagnosis=='Lung Squamous Cell Carcinoma', neutro_rna]\
                                                    .mean(axis=0).plot.hist(bins=5, histtype='step', ax=ax[1], label='SCC')

df_erv['RNAex'].loc[df_erv['RNAex'].Diagnosis=='Lung Adenocarcinoma', tcell_rna]\
                                                    .mean(axis=0).plot.hist(bins=5, histtype='step', ax=ax[2], label='Adeno')
df_erv['RNAex'].loc[df_erv['RNAex'].Diagnosis=='Lung Squamous Cell Carcinoma', tcell_rna]\
                                                    .mean(axis=0).plot.hist(bins=5, histtype='step', ax=ax[2], label='SCC')

ax[0].set_title('Mean RNAexp values for ERV')
ax[1].set_title('Mean RNAexp values for neutrophils')
ax[2].set_title('Mean RNAexp values for t-cells')
ax[0].legend(loc=1)
ax[1].legend(loc=1)
ax[2].legend(loc=2)

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(18,5))
df_erv['RNAex'].loc[df_erv['RNAex'].RTT==0, erv_rna]\
                                                    .mean(axis=0).plot.hist(bins=5, histtype='step', ax=ax[0], label='NR')
df_erv['RNAex'].loc[df_erv['RNAex'].RTT==1, erv_rna]\
                                                    .mean(axis=0).plot.hist(bins=5, histtype='step', ax=ax[0], label='R')

df_erv['RNAex'].loc[df_erv['RNAex'].RTT==0, neutro_rna]\
                                                    .mean(axis=0).plot.hist(bins=5, histtype='step', ax=ax[1], label='NR')
df_erv['RNAex'].loc[df_erv['RNAex'].RTT==1, neutro_rna]\
                                                    .mean(axis=0).plot.hist(bins=5, histtype='step', ax=ax[1], label='R')

df_erv['RNAex'].loc[df_erv['RNAex'].RTT==0, tcell_rna]\
                                                    .mean(axis=0).plot.hist(bins=5, histtype='step', ax=ax[2], label='NR')
df_erv['RNAex'].loc[df_erv['RNAex'].RTT==1, tcell_rna]\
                                                    .mean(axis=0).plot.hist(bins=5, histtype='step', ax=ax[2], label='R')

ax[0].set_title('Mean RNAexp values for ERV')
ax[1].set_title('Mean RNAexp values for neutrophils')
ax[2].set_title('Mean RNAexp values for t-cells')
ax[0].legend(loc=1)
ax[1].legend(loc=1)
ax[2].legend(loc=2)

In [ ]:
# relationship between ERV/T-cell/Neutrophils

In [ ]:
# Graph embeddings 

# Patient-to-patient clusters
## try MCL with Hadamard product and sum of logs

# ICA/CCA